In [2]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB4 import *

learning_rate = 0.0001  
gamma = 1
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,9)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 8)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"nomorspt.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "nomorspt.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("nomorspt.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

C:\Users\parkh\Anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



--------------------------------------------------
flow time: 53109, util : 0.843, makespan : 881
n_episode: 0, score : -5261.0, n_buffer : 340, eps : 8.0%
--------------------------------------------------
flow time: 52114, util : 0.844, makespan : 909
n_episode: 1, score : -5417.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 53109, util : 0.843, makespan : 881
n_episode: 1, score : -5261.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 48419, util : 0.848, makespan : 863
n_episode: 2, score : -5143.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 24710, util : 0.905, makespan : 738
n_episode: 2, score : -4088.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 26922, util : 0.886, makespan : 746
n_episode: 3, score : -4206.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time:

--------------------------------------------------
flow time: 31747, util : 0.944, makespan : 874
n_episode: 26, score : -5163.0, n_buffer : 9180, eps : 7.7%
--------------------------------------------------
flow time: 33219, util : 0.928, makespan : 869
n_episode: 27, score : -5154.0, n_buffer : 9520, eps : 7.7%
--------------------------------------------------
flow time: 28682, util : 0.936, makespan : 834
n_episode: 27, score : -4920.0, n_buffer : 9520, eps : 7.7%
--------------------------------------------------
flow time: 28303, util : 0.939, makespan : 880
n_episode: 28, score : -4965.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 27305, util : 0.938, makespan : 821
n_episode: 28, score : -4825.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 28772, util : 0.910, makespan : 814
n_episode: 29, score : -4822.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------


--------------------------------------------------
flow time: 48635, util : 0.856, makespan : 889
n_episode: 52, score : -5233.0, n_buffer : 18020, eps : 7.5%
--------------------------------------------------
flow time: 50189, util : 0.854, makespan : 901
n_episode: 53, score : -5326.0, n_buffer : 18360, eps : 7.5%
--------------------------------------------------
flow time: 55629, util : 0.845, makespan : 921
n_episode: 53, score : -5500.0, n_buffer : 18360, eps : 7.5%
--------------------------------------------------
flow time: 55568, util : 0.843, makespan : 935
n_episode: 54, score : -5567.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 41170, util : 0.887, makespan : 794
n_episode: 54, score : -4658.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 40850, util : 0.884, makespan : 781
n_episode: 55, score : -4621.0, n_buffer : 19040, eps : 7.4%
----------------------------------------------

--------------------------------------------------
flow time: 26585, util : 0.903, makespan : 759
n_episode: 78, score : -4204.0, n_buffer : 26860, eps : 7.2%
--------------------------------------------------
flow time: 29133, util : 0.903, makespan : 777
n_episode: 79, score : -4283.0, n_buffer : 27200, eps : 7.2%
--------------------------------------------------
flow time: 27226, util : 0.912, makespan : 805
n_episode: 79, score : -4552.0, n_buffer : 27200, eps : 7.2%
--------------------------------------------------
flow time: 29207, util : 0.886, makespan : 800
n_episode: 80, score : -4644.0, n_buffer : 27540, eps : 7.2%
--------------------------------------------------
flow time: 27615, util : 0.895, makespan : 737
n_episode: 80, score : -4382.0, n_buffer : 27540, eps : 7.2%
--------------------------------------------------
flow time: 29673, util : 0.882, makespan : 767
n_episode: 81, score : -4569.0, n_buffer : 27880, eps : 7.2%
----------------------------------------------

...

--------------------------------------------------
flow time: 24293, util : 0.919, makespan : 732
n_episode: 898, score : -4026.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24371, util : 0.919, makespan : 737
n_episode: 899, score : -4094.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24089, util : 0.908, makespan : 763
n_episode: 899, score : -4225.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24712, util : 0.911, makespan : 737
n_episode: 900, score : -4234.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24739, util : 0.909, makespan : 719
n_episode: 900, score : -4092.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24739, util : 0.908, makespan : 715
n_episode: 901, score : -4096.0, n_buffer : 100000, eps : 1.0%
----------------------------------

--------------------------------------------------
flow time: 24688, util : 0.903, makespan : 745
n_episode: 924, score : -4126.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24710, util : 0.905, makespan : 738
n_episode: 924, score : -4088.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24835, util : 0.902, makespan : 767
n_episode: 925, score : -4091.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 38591, util : 0.868, makespan : 824
n_episode: 925, score : -4858.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 40873, util : 0.865, makespan : 837
n_episode: 926, score : -4974.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24438, util : 0.903, makespan : 720
n_episode: 926, score : -4129.0, n_buffer : 100000, eps : 1.0%
----------------------------------

--------------------------------------------------
flow time: 26121, util : 0.908, makespan : 748
n_episode: 949, score : -4346.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 28194, util : 0.897, makespan : 764
n_episode: 950, score : -4473.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 31287, util : 0.884, makespan : 781
n_episode: 950, score : -4631.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 31016, util : 0.877, makespan : 781
n_episode: 951, score : -4611.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 40058, util : 0.866, makespan : 868
n_episode: 951, score : -5074.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 40025, util : 0.871, makespan : 865
n_episode: 952, score : -5097.0, n_buffer : 100000, eps : 1.0%
----------------------------------

--------------------------------------------------
flow time: 24584, util : 0.926, makespan : 706
n_episode: 975, score : -4090.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24935, util : 0.927, makespan : 734
n_episode: 975, score : -4357.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 25104, util : 0.918, makespan : 725
n_episode: 976, score : -4327.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 25047, util : 0.925, makespan : 731
n_episode: 976, score : -4337.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 26292, util : 0.925, makespan : 748
n_episode: 977, score : -4394.0, n_buffer : 100000, eps : 1.0%
--------------------------------------------------
flow time: 24592, util : 0.904, makespan : 727
n_episode: 977, score : -4200.0, n_buffer : 100000, eps : 1.0%
----------------------------------

FlowTime: 24569
machine_util: 0
util: 0.9156853509185116
makespan: 758
Score -4246.0
FlowTime: 24569
machine_util: 0
util: 0.9156853509185116
makespan: 758
Score -4246.0


In [ ]:


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
import collections
import random
from FAB4 import *

learning_rate = 0.0001  
gamma = 0.98
buffer_limit = 100000
batch_size = 64

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit);
    def put(self, transition):
        self.buffer.append(transition)
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [],[],[],[],[]
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])
            
        return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(64,64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,9)
    
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0, 8)
        else:
            return out.argmax().item()
        
    def select_action(self, obs, epsilon):
        out = self.forward(obs)
        return out.argmax().item(),out
        
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)
        #q.number_of_time_list[a] += 1    
        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max (1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
def main():
    env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1)
    q = Qnet()
    q_target = Qnet()
    q_target.load_state_dict(q.state_dict())
    memory = ReplayBuffer()
    print_interval = 1
    q_load = 10
    score = 0.0
    optimizer = optim.Adam(q.parameters(), lr=learning_rate)
    
    for n_epi in range(1000):
        #여기는 sample_action 구간
        epsilon = max(0.01 , 0.08 - 0.02*(n_epi/200))
        s = env.reset()
        done = False
        score = 0.0
        while not done:
            a = q.sample_action(torch.from_numpy(s). float(), epsilon)
            s_prime, r, done = env.step(a)
            done_mask =0.0 if done else 1.0
            if done == False:
                memory.put((s,a,r,s_prime,done_mask))
                s = s_prime
                score += r
            if done:
                break
            
        #학습구간    
        if memory.size()>1000:
            train(q, q_target, memory, optimizer)
        
        #결과 및 파라미터 저장    
        if n_epi % print_interval==0 and n_epi!=0:
            #q_target.load_state_dict(q.state_dict())
            params = q.state_dict()
            param_name = str(n_epi)+"nomorspt.pt"
            #print(param_name)
            torch.save(params, param_name)
            Flow_time, machine_utill ,util, makespan = env.performance_measure()
            print("--------------------------------------------------")
            print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
            print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
            #score=0.0
        
        #여기는 select_action 구간
        s = env.reset()
        done = False
        score = 0.0
        params = q.state_dict()
        torch.save(params, "nomorspt.pt" )
        while not done:
            a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
            #print(a_list)
            #print(a)
            s_prime, r, done = env.step(a)
            #print(r)
            s = s_prime
            score += r
            if done:
                break
        Flow_time, machine_utill ,util, makespan = env.performance_measure()
        print("--------------------------------------------------")
        print("flow time: {}, util : {:.3f}, makespan : {}".format(Flow_time, util, makespan))
        print("n_episode: {}, score : {:.1f}, n_buffer : {}, eps : {:.1f}%".format(n_epi, score/print_interval,memory.size(),epsilon*100))
        
        
        
        if n_epi % q_load ==0 and n_epi!=0:
            q_target.load_state_dict(q.state_dict())
    
    
    s = env.reset()
    done = False
    score = 0.0
    while not done:
        a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
        #print(a_list)
        #print(a)
        s_prime, r, done = env.step(a)
        #print(r)
        s = s_prime
        score += r
        if done:
            break
    Flow_time, machine_util, util, makespan = env.performance_measure()
    env.gannt_chart()
    return Flow_time, machine_util, util, makespan, score
for i in range(1):
    Flow_time, machine_util, util, makespan, score =main()
    print("FlowTime:" , Flow_time)
    print("machine_util:" , machine_util)
    print("util:" , util)
    print("makespan:" , makespan)
    print("Score" , score)
    
    
params = torch.load("nomorspt.pt")
q = Qnet()
q.load_state_dict(params)
q.eval()
env = FJSP_simulator('C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_SIM7_all.csv','C:/Users/parkh/FJSP_SETUP_SIM.csv',"C:/Users/parkh/git_tlsgudcks/simulator/data/FJSP_Fab.csv",1) 
s = env.reset()
done = False
score = 0.0
epsilon = max(0.01 , 0.08 - 0.02*(20/200))
while not done:
    a, a_list = q.select_action(torch.from_numpy(s). float(), epsilon)
    #print(a_list)
    #print(a)
    s_prime, r, done = env.step(a)
    #print(r)
    s = s_prime
    score += r
    if done:
        break
Flow_time, machine_util, util, makespan = env.performance_measure()
print("FlowTime:" , Flow_time)
print("machine_util:" , machine_util)
print("util:" , util)
print("makespan:" , makespan)
print("Score" , score)    

--------------------------------------------------
flow time: 25926, util : 0.937, makespan : 733
n_episode: 0, score : -8539.0, n_buffer : 340, eps : 8.0%
--------------------------------------------------
flow time: 26640, util : 0.934, makespan : 756
n_episode: 1, score : -8794.0, n_buffer : 680, eps : 8.0%
--------------------------------------------------
flow time: 25926, util : 0.937, makespan : 733
n_episode: 1, score : -8539.0, n_buffer : 680, eps : 8.0%


<ipython-input-1-a814097aa090>:33: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:210.)
  return torch.tensor(s_lst, dtype=torch. float),torch.tensor(a_lst), torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch. float), torch.tensor(done_mask_lst)


--------------------------------------------------
flow time: 27756, util : 0.920, makespan : 751
n_episode: 2, score : -8901.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 27204, util : 0.934, makespan : 770
n_episode: 2, score : -8877.0, n_buffer : 1020, eps : 8.0%
--------------------------------------------------
flow time: 29637, util : 0.913, makespan : 730
n_episode: 3, score : -9081.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 51142, util : 0.858, makespan : 907
n_episode: 3, score : -11059.0, n_buffer : 1360, eps : 8.0%
--------------------------------------------------
flow time: 48354, util : 0.857, makespan : 904
n_episode: 4, score : -11116.0, n_buffer : 1700, eps : 8.0%
--------------------------------------------------
flow time: 48064, util : 0.860, makespan : 921
n_episode: 4, score : -11222.0, n_buffer : 1700, eps : 8.0%
--------------------------------------------------
flow

--------------------------------------------------
flow time: 35519, util : 0.884, makespan : 761
n_episode: 28, score : -9256.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 50200, util : 0.849, makespan : 867
n_episode: 28, score : -10291.0, n_buffer : 9860, eps : 7.7%
--------------------------------------------------
flow time: 52394, util : 0.849, makespan : 893
n_episode: 29, score : -10545.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------
flow time: 45936, util : 0.863, makespan : 831
n_episode: 29, score : -9936.0, n_buffer : 10200, eps : 7.7%
--------------------------------------------------
flow time: 49002, util : 0.853, makespan : 865
n_episode: 30, score : -10345.0, n_buffer : 10540, eps : 7.7%
--------------------------------------------------
flow time: 24566, util : 0.932, makespan : 708
n_episode: 30, score : -8055.0, n_buffer : 10540, eps : 7.7%
---------------------------------------------

--------------------------------------------------
flow time: 39056, util : 0.913, makespan : 878
n_episode: 54, score : -10590.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 27668, util : 0.920, makespan : 806
n_episode: 54, score : -10105.0, n_buffer : 18700, eps : 7.5%
--------------------------------------------------
flow time: 28640, util : 0.918, makespan : 813
n_episode: 55, score : -10009.0, n_buffer : 19040, eps : 7.4%
--------------------------------------------------
flow time: 32126, util : 0.939, makespan : 846
n_episode: 55, score : -9981.0, n_buffer : 19040, eps : 7.4%
--------------------------------------------------
flow time: 33125, util : 0.926, makespan : 851
n_episode: 56, score : -10386.0, n_buffer : 19380, eps : 7.4%
--------------------------------------------------
flow time: 38020, util : 0.919, makespan : 873
n_episode: 56, score : -10454.0, n_buffer : 19380, eps : 7.4%
-----------------------------------------

...

--------------------------------------------------
flow time: 23973, util : 0.915, makespan : 718
n_episode: 466, score : -8459.0, n_buffer : 100000, eps : 3.3%
--------------------------------------------------
flow time: 25039, util : 0.934, makespan : 747
n_episode: 467, score : -8585.0, n_buffer : 100000, eps : 3.3%
--------------------------------------------------
flow time: 23946, util : 0.939, makespan : 705
n_episode: 467, score : -8335.0, n_buffer : 100000, eps : 3.3%
--------------------------------------------------
flow time: 24792, util : 0.928, makespan : 724
n_episode: 468, score : -8261.0, n_buffer : 100000, eps : 3.3%
--------------------------------------------------
flow time: 24483, util : 0.910, makespan : 717
n_episode: 468, score : -8214.0, n_buffer : 100000, eps : 3.3%
--------------------------------------------------
flow time: 24458, util : 0.917, makespan : 760
n_episode: 469, score : -8608.0, n_buffer : 100000, eps : 3.3%
----------------------------------

--------------------------------------------------
flow time: 24685, util : 0.949, makespan : 841
n_episode: 492, score : -10371.0, n_buffer : 100000, eps : 3.1%
--------------------------------------------------
flow time: 24771, util : 0.913, makespan : 738
n_episode: 492, score : -8670.0, n_buffer : 100000, eps : 3.1%
--------------------------------------------------
flow time: 24931, util : 0.924, makespan : 763
n_episode: 493, score : -8891.0, n_buffer : 100000, eps : 3.1%
--------------------------------------------------
flow time: 24294, util : 0.913, makespan : 721
n_episode: 493, score : -8164.0, n_buffer : 100000, eps : 3.1%
--------------------------------------------------
flow time: 24572, util : 0.907, makespan : 722
n_episode: 494, score : -8586.0, n_buffer : 100000, eps : 3.1%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_episode: 494, score : -8440.0, n_buffer : 100000, eps : 3.1%
---------------------------------

--------------------------------------------------
flow time: 23971, util : 0.932, makespan : 727
n_episode: 517, score : -8313.0, n_buffer : 100000, eps : 2.8%
--------------------------------------------------
flow time: 23722, util : 0.940, makespan : 750
n_episode: 518, score : -8449.0, n_buffer : 100000, eps : 2.8%
--------------------------------------------------
flow time: 24141, util : 0.917, makespan : 719
n_episode: 518, score : -8192.0, n_buffer : 100000, eps : 2.8%
--------------------------------------------------
flow time: 24280, util : 0.914, makespan : 751
n_episode: 519, score : -8835.0, n_buffer : 100000, eps : 2.8%
--------------------------------------------------
flow time: 24024, util : 0.934, makespan : 713
n_episode: 519, score : -8254.0, n_buffer : 100000, eps : 2.8%
--------------------------------------------------
flow time: 25128, util : 0.929, makespan : 731
n_episode: 520, score : -8659.0, n_buffer : 100000, eps : 2.8%
----------------------------------

--------------------------------------------------
flow time: 24332, util : 0.927, makespan : 746
n_episode: 543, score : -8628.0, n_buffer : 100000, eps : 2.6%
--------------------------------------------------
flow time: 24162, util : 0.914, makespan : 723
n_episode: 543, score : -8090.0, n_buffer : 100000, eps : 2.6%
--------------------------------------------------
flow time: 24442, util : 0.897, makespan : 749
n_episode: 544, score : -8722.0, n_buffer : 100000, eps : 2.6%
--------------------------------------------------
flow time: 24320, util : 0.917, makespan : 745
n_episode: 544, score : -8432.0, n_buffer : 100000, eps : 2.6%
--------------------------------------------------
flow time: 24850, util : 0.911, makespan : 714
n_episode: 545, score : -8445.0, n_buffer : 100000, eps : 2.6%
--------------------------------------------------
flow time: 24021, util : 0.941, makespan : 740
n_episode: 545, score : -8352.0, n_buffer : 100000, eps : 2.6%
----------------------------------

--------------------------------------------------
flow time: 24021, util : 0.941, makespan : 740
n_episode: 568, score : -8352.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24022, util : 0.941, makespan : 743
n_episode: 569, score : -8469.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24438, util : 0.913, makespan : 733
n_episode: 569, score : -8595.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24930, util : 0.912, makespan : 743
n_episode: 570, score : -8661.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 23973, util : 0.942, makespan : 740
n_episode: 570, score : -8440.0, n_buffer : 100000, eps : 2.3%
--------------------------------------------------
flow time: 24244, util : 0.941, makespan : 738
n_episode: 571, score : -8501.0, n_buffer : 100000, eps : 2.3%
----------------------------------